In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 
from warnings import simplefilter
import pandas as pd

In [2]:
import models
import class_sampling
import train
import metric_utils
import inference

In [3]:
NUM_CLASSES = 10
n_epochs = 50
batch_size_train = 64
batch_size_test = 1000
momentum = 0



random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

NUM_CLASSES_REDUCED = 2
nums = (0, 1)
ratio = (100, 1)
#nums = (0, 3, 1)
#ratio = (200, 20, 1)

CLASS_LABELS = {'airplane': 0,
                 'automobile': 1,
                 'bird': 2,
                 'cat': 3,
                 'deer': 4,
                 'dog': 5,
                 'frog': 6,
                 'horse': 7,
                 'ship': 8,
                 'truck': 9}


simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

In [4]:
# name, num_classes, classes used, ratio, learning rate, mean 10, variance 10, mean 20, variance 20, ... 50

# mean, variance every 10 epochs - average of 10 models 
# name, num_classes, classes used, ratio, learning rate, mean 10, variance 10, mean 20, variance 20, ... 50
# name: normal/ratio/oversampled/undersampled/weighted

col_names = ["name", 
            "num_classes", 
            "classes_used", 
            "ratio", 
            "learning_rate", 
            "mean_0", "variance_0",
            "mean_10", "variance_10",
            "mean_20", "variance_20",
            "mean_30", "variance_30",
            "mean_40", "variance_40",
            "mean_50", "variance_50"]

rows = []

In [5]:
train_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor() ]))  


test_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()]))

train_CIFAR10.data = train_CIFAR10.data.reshape(50000, 3, 32, 32)
test_CIFAR10.data = test_CIFAR10.data.reshape(10000, 3, 32, 32)

    
reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
targets = ratio_train_CIFAR10.labels 

class_count = np.unique(targets, return_counts=True)[1]

# try undersampling 

weight = 1. / class_count

samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
# how does this work? 

# undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=False)

weight *= min(class_count)

In [7]:
train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

In [8]:
print(train_loader_oversampled.dataset.labels.shape)

torch.Size([5050])


In [9]:
# SIGMOID 2 CLASS normal AUC saving  
# df.to_csv()

momentum=0
learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SigmoidLogisticRegression(NUM_CLASSES_REDUCED, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)
   


Test set: Avg. loss: 0.02157968044281006, AUC: 0.6867034999999999


Test set: Avg. loss: 0.03557538032531738, AUC: 0.570759


Test set: Avg. loss: 0.013752534389495849, AUC: 0.889133


Test set: Avg. loss: 0.007762441396713257, AUC: 0.8451270000000001


Test set: Avg. loss: 0.008736375808715821, AUC: 0.8208995


Test set: Avg. loss: 0.008814629077911376, AUC: 0.8945854999999999


Test set: Avg. loss: 0.023658716201782226, AUC: 0.398378


Test set: Avg. loss: 0.12261966705322265, AUC: 0.532487


Test set: Avg. loss: 0.00910294771194458, AUC: 0.8903105


Test set: Avg. loss: 0.00748134970664978, AUC: 0.8410195


Test set: Avg. loss: 0.048084203720092776, AUC: 0.7297494999999999


Test set: Avg. loss: 0.007408306837081909, AUC: 0.848622


Test set: Avg. loss: 0.04466411209106445, AUC: 0.5848144999999999


Test set: Avg. loss: 0.009844433307647706, AUC: 0.8901745


Test set: Avg. loss: 0.011238266944885254, AUC: 0.7808945


Test set: Avg. loss: 0.010974647521972657, AUC: 0.896553500000000


Test set: Avg. loss: 0.0027123184204101564, AUC: 0.800652


Test set: Avg. loss: 0.0023746830224990845, AUC: 0.7995225


Test set: Avg. loss: 0.07314897918701171, AUC: 0.499006


Test set: Avg. loss: 0.0036382440328598022, AUC: 0.7799014999999999


Test set: Avg. loss: 0.0028783069849014284, AUC: 0.7964144999999999


Test set: Avg. loss: 0.002862035274505615, AUC: 0.814852


Test set: Avg. loss: 0.002274002432823181, AUC: 0.8088519999999999


Test set: Avg. loss: 0.0021738073825836183, AUC: 0.8198215000000001


Test set: Avg. loss: 0.02219521713256836, AUC: 0.616943


Test set: Avg. loss: 0.003909741163253784, AUC: 0.788864


Test set: Avg. loss: 0.003301565766334534, AUC: 0.807256


Test set: Avg. loss: 0.002738043785095215, AUC: 0.795409


Test set: Avg. loss: 0.0024630810022354126, AUC: 0.8134885


Test set: Avg. loss: 0.0022289674282073976, AUC: 0.813447


Test set: Avg. loss: 0.024299232482910157, AUC: 0.6556175


Test set: Avg. loss: 0.0040311203002929685, AUC: 0.7867195


Test 


Test set: Avg. loss: 0.009326772212982178, AUC: 0.4918055


Test set: Avg. loss: 0.008536043167114258, AUC: 0.5214624999999999


Test set: Avg. loss: 0.007923449277877808, AUC: 0.5457095000000001


Test set: Avg. loss: 0.03852338981628418, AUC: 0.5131654999999999


Test set: Avg. loss: 0.00857986831665039, AUC: 0.72758


Test set: Avg. loss: 0.00751304292678833, AUC: 0.7379820000000001


Test set: Avg. loss: 0.006684453248977661, AUC: 0.7481395


Test set: Avg. loss: 0.006065703630447387, AUC: 0.756637


Test set: Avg. loss: 0.005620069503784179, AUC: 0.761727


Test set: Avg. loss: 0.06580369186401368, AUC: 0.5360860000000001


Test set: Avg. loss: 0.01131051778793335, AUC: 0.40780099999999997


Test set: Avg. loss: 0.009915541172027588, AUC: 0.446294


Test set: Avg. loss: 0.00885139274597168, AUC: 0.48339750000000004


Test set: Avg. loss: 0.008074794054031373, AUC: 0.5144225


Test set: Avg. loss: 0.007473847150802612, AUC: 0.543707


Test set: Avg. loss: 0.034681669235229494, AUC

In [10]:
print(learning_rate_aucs.shape)
auc_mean = np.mean(learning_rate_aucs, axis=1)
print(auc_mean.shape)
auc_variance = np.var(learning_rate_aucs, axis=1)
print(auc_variance.shape)

for i in range(len(learning_rates)): 
    row = ["normal", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)

print(rows)

# pd.DataFrame(rows, columns = col_names) 

(5, 10, 6)
(5, 6)
(5, 6)
[['normal', 2, (0, 1), (1, 1), 1e-05, 0.5483786499999999, 0.009423741928102494, 0.77910575, 0.014506833817462497, 0.84148475, 0.0023757573245125002, 0.8049157000000001, 0.004259281018710005, 0.7472957000000001, 0.016494224985460004, 0.8236022000000001, 0.01086748083321], ['normal', 2, (0, 1), (1, 1), 1e-06, 0.53490505, 0.008606034898972499, 0.7956359500000001, 0.00018409845702250138, 0.8118000500000001, 0.0003450463902725021, 0.8264205500000001, 0.00012591282747250035, 0.8233665499999999, 0.00017876096052249927, 0.8326911499999999, 0.00016672283585250023], ['normal', 2, (0, 1), (1, 1), 5e-07, 0.5275359, 0.008331209211690001, 0.77333865, 0.00013976534670250107, 0.7887498, 0.00020414396776000026, 0.8012477, 0.00014091100445999934, 0.8088807499999999, 7.449802441249995e-05, 0.81046945, 5.3778948072500036e-05], ['normal', 2, (0, 1), (1, 1), 1e-07, 0.5117162999999999, 0.005766031790659998, 0.6952516999999999, 0.00021697121525999984, 0.7379711999999999, 0.00010511842

In [11]:
# SIGMOID 2 CLASS ratio

momentum=0
learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SigmoidLogisticRegression(2, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.02125685691833496, AUC: 0.602456


Test set: Avg. loss: 0.012000832557678223, AUC: 0.7643120000000001


Test set: Avg. loss: 0.009370763778686524, AUC: 0.7785015


Test set: Avg. loss: 0.006626487970352173, AUC: 0.7748189999999999


Test set: Avg. loss: 0.0075504629611969, AUC: 0.800147


Test set: Avg. loss: 0.012373299121856689, AUC: 0.8178645


Test set: Avg. loss: 0.02702017593383789, AUC: 0.47021199999999996


Test set: Avg. loss: 0.016043559074401855, AUC: 0.7742609999999999


Test set: Avg. loss: 0.015801690101623536, AUC: 0.7990839999999999


Test set: Avg. loss: 0.005397058248519897, AUC: 0.77552


Test set: Avg. loss: 0.010859752655029297, AUC: 0.8051385


Test set: Avg. loss: 0.014050290107727051, AUC: 0.8125285


Test set: Avg. loss: 0.018543965339660646, AUC: 0.4351165


Test set: Avg. loss: 0.011069374561309814, AUC: 0.7017090000000001


Test set: Avg. loss: 0.028241270065307616, AUC: 0.7887615


Test set: Avg. loss: 0.011797702312469483, AUC: 0.77


Test set: Avg. loss: 0.019146682739257813, AUC: 0.6882010000000001


Test set: Avg. loss: 0.03985671997070313, AUC: 0.6524395000000001


Test set: Avg. loss: 0.029123291969299317, AUC: 0.6751805


Test set: Avg. loss: 0.0236292667388916, AUC: 0.685955


Test set: Avg. loss: 0.021259888648986817, AUC: 0.6934115000000001


Test set: Avg. loss: 0.01984457778930664, AUC: 0.6958280000000001


Test set: Avg. loss: 0.018520832061767578, AUC: 0.6977819999999999


Test set: Avg. loss: 0.024677947998046876, AUC: 0.7387680000000001


Test set: Avg. loss: 0.02659729290008545, AUC: 0.756372


Test set: Avg. loss: 0.02425697326660156, AUC: 0.762513


Test set: Avg. loss: 0.022470035552978516, AUC: 0.767377


Test set: Avg. loss: 0.02074063682556152, AUC: 0.7683085000000001


Test set: Avg. loss: 0.019403928756713866, AUC: 0.7677560000000001


Test set: Avg. loss: 0.06291898155212403, AUC: 0.498011


Test set: Avg. loss: 0.025504664421081544, AUC: 0.7104514999999999


Test set: Avg. loss: 0.02322420


Test set: Avg. loss: 0.032610072135925296, AUC: 0.6372635


Test set: Avg. loss: 0.0335287914276123, AUC: 0.6393449999999999


Test set: Avg. loss: 0.041125255584716795, AUC: 0.6636755


Test set: Avg. loss: 0.04052045631408691, AUC: 0.6646285000000001


Test set: Avg. loss: 0.039974874496459964, AUC: 0.6644275


Test set: Avg. loss: 0.03947804641723633, AUC: 0.6651575


Test set: Avg. loss: 0.03902339172363281, AUC: 0.666705


Test set: Avg. loss: 0.03860315895080566, AUC: 0.665115


Test set: Avg. loss: 0.05078658676147461, AUC: 0.47564100000000004


Test set: Avg. loss: 0.018972426414489745, AUC: 0.6393065


Test set: Avg. loss: 0.02271225452423096, AUC: 0.6556365


Test set: Avg. loss: 0.024325194358825682, AUC: 0.662029


Test set: Avg. loss: 0.025279623985290527, AUC: 0.6652765


Test set: Avg. loss: 0.025866005897521974, AUC: 0.6666974999999999


Test set: Avg. loss: 0.07260027313232421, AUC: 0.5614969999999999


Test set: Avg. loss: 0.07098178482055664, AUC: 0.5678199999999999

In [12]:
# SIGMOID 2 CLASS oversampled

momentum=0
learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SigmoidLogisticRegression(2, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.01637674903869629, AUC: 0.4367375


Test set: Avg. loss: 0.010641933917999268, AUC: 0.8395484999999999


Test set: Avg. loss: 0.013783086776733398, AUC: 0.8422505


Test set: Avg. loss: 0.017524030685424803, AUC: 0.8291925000000001


Test set: Avg. loss: 0.017428361892700195, AUC: 0.8318459999999999


Test set: Avg. loss: 0.024435037612915038, AUC: 0.8188295


Test set: Avg. loss: 0.01486136531829834, AUC: 0.46075949999999993


Test set: Avg. loss: 0.032964454650878904, AUC: 0.7955770000000001


Test set: Avg. loss: 0.015161131858825684, AUC: 0.8475695000000001


Test set: Avg. loss: 0.015921735286712646, AUC: 0.836827


Test set: Avg. loss: 0.014466152667999268, AUC: 0.8382930000000001


Test set: Avg. loss: 0.020693782806396485, AUC: 0.8360789999999999


Test set: Avg. loss: 0.022908369064331056, AUC: 0.4506915


Test set: Avg. loss: 0.009568213939666748, AUC: 0.8230355


Test set: Avg. loss: 0.010080985069274902, AUC: 0.813942


Test set: Avg. loss: 0.0190989


Test set: Avg. loss: 0.005539389848709106, AUC: 0.7674725000000001


Test set: Avg. loss: 0.005486319065093994, AUC: 0.7705655


Test set: Avg. loss: 0.005469388008117676, AUC: 0.772327


Test set: Avg. loss: 0.00989992666244507, AUC: 0.611619


Test set: Avg. loss: 0.0061765162944793705, AUC: 0.7168685000000001


Test set: Avg. loss: 0.005567391872406006, AUC: 0.731827


Test set: Avg. loss: 0.005605482816696167, AUC: 0.7435685


Test set: Avg. loss: 0.005507049083709717, AUC: 0.7476005


Test set: Avg. loss: 0.005775624275207519, AUC: 0.7563095


Test set: Avg. loss: 0.016201416015625, AUC: 0.6161665000000001


Test set: Avg. loss: 0.0049600756168365475, AUC: 0.7431115


Test set: Avg. loss: 0.005048235654830932, AUC: 0.772035


Test set: Avg. loss: 0.004934582710266113, AUC: 0.7816319999999999


Test set: Avg. loss: 0.0049291555881500245, AUC: 0.787601


Test set: Avg. loss: 0.005209622383117676, AUC: 0.7898075


Test set: Avg. loss: 0.032729392051696775, AUC: 0.687705


Test set: 


Test set: Avg. loss: 0.009883739948272706, AUC: 0.546036


Test set: Avg. loss: 0.009436863899230957, AUC: 0.560061


Test set: Avg. loss: 0.00903068733215332, AUC: 0.57339


Test set: Avg. loss: 0.008698916912078858, AUC: 0.5871270000000001


Test set: Avg. loss: 0.008426181316375733, AUC: 0.5969420000000001


Test set: Avg. loss: 0.014684648990631103, AUC: 0.4036685


Test set: Avg. loss: 0.013852731227874756, AUC: 0.41796150000000004


Test set: Avg. loss: 0.012945362567901611, AUC: 0.434597


Test set: Avg. loss: 0.012135267734527588, AUC: 0.4547225


Test set: Avg. loss: 0.011454725742340088, AUC: 0.471992


Test set: Avg. loss: 0.010855827331542969, AUC: 0.48780100000000004


Test set: Avg. loss: 0.022012916564941407, AUC: 0.6393645


Test set: Avg. loss: 0.008470763683319091, AUC: 0.560241


Test set: Avg. loss: 0.007950004816055298, AUC: 0.580861


Test set: Avg. loss: 0.007570127725601197, AUC: 0.5972609999999999


Test set: Avg. loss: 0.007271449089050293, AUC: 0.6124415


T

In [13]:
# SIGMOID 2 CLASS weighted 
# did I do the weighting correctly

momentum=0
learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SigmoidLogisticRegression(2, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False, class_weights=weight)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["weighted", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.05379483413696289, AUC: 0.629664


Test set: Avg. loss: 0.07637473678588867, AUC: 0.5569495


Test set: Avg. loss: 0.06947299957275391, AUC: 0.571782


Test set: Avg. loss: 0.06694623947143555, AUC: 0.579026


Test set: Avg. loss: 0.06513669204711914, AUC: 0.5818414999999999


Test set: Avg. loss: 0.06292413902282715, AUC: 0.59243


Test set: Avg. loss: 0.02169177055358887, AUC: 0.4096525000000001


Test set: Avg. loss: 0.20370823669433594, AUC: 0.4995


Test set: Avg. loss: 0.18655671691894532, AUC: 0.4995


Test set: Avg. loss: 0.1693976516723633, AUC: 0.4995


Test set: Avg. loss: 0.15224893188476563, AUC: 0.5004989999999999


Test set: Avg. loss: 0.13508262634277343, AUC: 0.5024959999999999


Test set: Avg. loss: 0.06560427856445313, AUC: 0.4850385


Test set: Avg. loss: 0.18498682403564454, AUC: 0.4995


Test set: Avg. loss: 0.167802490234375, AUC: 0.4995


Test set: Avg. loss: 0.15062786102294923, AUC: 0.4995


Test set: Avg. loss: 0.13346968841552734, AUC


Test set: Avg. loss: 0.04469722938537598, AUC: 0.6374625


Test set: Avg. loss: 0.04914028739929199, AUC: 0.617496


Test set: Avg. loss: 0.05085527992248535, AUC: 0.608976


Test set: Avg. loss: 0.05152193450927734, AUC: 0.6069705


Test set: Avg. loss: 0.05176607513427734, AUC: 0.6057385000000001


Test set: Avg. loss: 0.06768664932250977, AUC: 0.46149950000000006


Test set: Avg. loss: 0.055133050918579105, AUC: 0.5396710000000001


Test set: Avg. loss: 0.058051408767700194, AUC: 0.5324420000000001


Test set: Avg. loss: 0.05838982391357422, AUC: 0.533898


Test set: Avg. loss: 0.05835397720336914, AUC: 0.533115


Test set: Avg. loss: 0.05825642204284668, AUC: 0.5331065


Test set: Avg. loss: 0.05640487861633301, AUC: 0.613344


Test set: Avg. loss: 0.06016583061218262, AUC: 0.5962255


Test set: Avg. loss: 0.06174511528015137, AUC: 0.589357


Test set: Avg. loss: 0.06272874259948731, AUC: 0.5838285000000001


Test set: Avg. loss: 0.06329341316223144, AUC: 0.583849


Test set: Avg.


Test set: Avg. loss: 0.01764606285095215, AUC: 0.6357470000000001


Test set: Avg. loss: 0.02182399845123291, AUC: 0.6516635


Test set: Avg. loss: 0.02445944595336914, AUC: 0.659654


Test set: Avg. loss: 0.026503061294555662, AUC: 0.6655044999999999


Test set: Avg. loss: 0.028111204147338868, AUC: 0.6678045000000001


Test set: Avg. loss: 0.015549704551696778, AUC: 0.428295


Test set: Avg. loss: 0.024327635765075684, AUC: 0.5713385


Test set: Avg. loss: 0.028587156295776368, AUC: 0.5898215000000001


Test set: Avg. loss: 0.03121313190460205, AUC: 0.598795


Test set: Avg. loss: 0.03314826202392578, AUC: 0.6051839999999999


Test set: Avg. loss: 0.03467856025695801, AUC: 0.608279


Test set: Avg. loss: 0.08861247634887695, AUC: 0.511401


Test set: Avg. loss: 0.08861773300170898, AUC: 0.511401


Test set: Avg. loss: 0.08862295913696289, AUC: 0.511402


Test set: Avg. loss: 0.08862812042236329, AUC: 0.511402


Test set: Avg. loss: 0.0886338005065918, AUC: 0.511402


Test set: Avg. 

In [14]:
NUM_CLASSES_REDUCED = 3
nums = (0, 3, 1)
ratio = (200, 20, 1)

reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums)
targets = ratio_train_CIFAR10.labels 

class_count = np.unique(targets, return_counts=True)[1]
weight = 1. / class_count
samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
weight *= min(class_count)

train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

In [15]:
# Softmax 3 class normal

learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SoftmaxLogisticRegression(NUM_CLASSES_REDUCED, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["normal", 3, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.08266295878092447, AUC: 0.39183724999999997


Test set: Avg. loss: 0.01976626205444336, AUC: 0.72925


Test set: Avg. loss: 0.058622213999430337, AUC: 0.56725


Test set: Avg. loss: 0.014332512219746907, AUC: 0.78425


Test set: Avg. loss: 0.019861311594645183, AUC: 0.7202500000000001


Test set: Avg. loss: 0.036045097351074216, AUC: 0.627


Test set: Avg. loss: 0.12215186564127604, AUC: 0.4864865


Test set: Avg. loss: 0.05162781016031901, AUC: 0.56925


Test set: Avg. loss: 0.015693854331970215, AUC: 0.789


Test set: Avg. loss: 0.08192098999023438, AUC: 0.52325


Test set: Avg. loss: 0.015575826327006022, AUC: 0.789


Test set: Avg. loss: 0.10853070576985677, AUC: 0.507


Test set: Avg. loss: 0.032606922149658206, AUC: 0.538401


Test set: Avg. loss: 0.0514029426574707, AUC: 0.57775


Test set: Avg. loss: 0.03534972763061523, AUC: 0.71075


Test set: Avg. loss: 0.05201171747843424, AUC: 0.6539999999999999


Test set: Avg. loss: 0.01641814613342285, AUC: 0.755


Test set: Avg. loss: 0.005277069727579752, AUC: 0.6344259999999999


Test set: Avg. loss: 0.004209813833236695, AUC: 0.63739325


Test set: Avg. loss: 0.0032180991967519124, AUC: 0.69158575


Test set: Avg. loss: 0.002938907305399577, AUC: 0.7007500000000001


Test set: Avg. loss: 0.00262983775138855, AUC: 0.673


Test set: Avg. loss: 0.057730695088704424, AUC: 0.5239729999999999


Test set: Avg. loss: 0.00428039534886678, AUC: 0.68925


Test set: Avg. loss: 0.00354465921719869, AUC: 0.7055


Test set: Avg. loss: 0.005011731783548991, AUC: 0.5762499999999999


Test set: Avg. loss: 0.0025783124764760337, AUC: 0.6844999999999999


Test set: Avg. loss: 0.0027911150455474853, AUC: 0.6487499999999999


Test set: Avg. loss: 0.04937011464436849, AUC: 0.49556500000000003


Test set: Avg. loss: 0.0050439596176147464, AUC: 0.6656072499999999


Test set: Avg. loss: 0.00390058167775472, AUC: 0.69


Test set: Avg. loss: 0.0036994036038716636, AUC: 0.63475


Test set: Avg. loss: 0.00293303600947062


Test set: Avg. loss: 0.008576404571533204, AUC: 0.60327


Test set: Avg. loss: 0.058335155487060544, AUC: 0.45527375000000003


Test set: Avg. loss: 0.012643045743306477, AUC: 0.512841


Test set: Avg. loss: 0.010624648412068685, AUC: 0.54435425


Test set: Avg. loss: 0.0093350830078125, AUC: 0.5715919999999999


Test set: Avg. loss: 0.008423053741455078, AUC: 0.6004125


Test set: Avg. loss: 0.007783047835032145, AUC: 0.61660475


Test set: Avg. loss: 0.02726526896158854, AUC: 0.43432925


Test set: Avg. loss: 0.016604571024576822, AUC: 0.4998475


Test set: Avg. loss: 0.01420715872446696, AUC: 0.533075


Test set: Avg. loss: 0.012450122197469075, AUC: 0.55655775


Test set: Avg. loss: 0.011127525965372722, AUC: 0.5804195


Test set: Avg. loss: 0.010122740109761556, AUC: 0.59665325


Test set: Avg. loss: 0.09527002461751302, AUC: 0.5012515


Test set: Avg. loss: 0.011262702624003092, AUC: 0.5803477499999999


Test set: Avg. loss: 0.00956979242960612, AUC: 0.5957509999999999


Test se

In [16]:
# Softmax 3 class ratio

learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SoftmaxLogisticRegression(NUM_CLASSES_REDUCED, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.07566189320882162, AUC: 0.499


Test set: Avg. loss: 0.04198944473266602, AUC: 0.6019669999999999


Test set: Avg. loss: 0.06919888814290365, AUC: 0.5429935


Test set: Avg. loss: 0.048214862823486325, AUC: 0.5885180000000001


Test set: Avg. loss: 0.044022106170654295, AUC: 0.5653637499999999


Test set: Avg. loss: 0.16341344706217448, AUC: 0.501


Test set: Avg. loss: 0.10342728678385417, AUC: 0.49042025


Test set: Avg. loss: 0.0685009511311849, AUC: 0.5324612500000001


Test set: Avg. loss: 0.18458396911621094, AUC: 0.501


Test set: Avg. loss: 0.02756735610961914, AUC: 0.628402


Test set: Avg. loss: 0.032471574147542315, AUC: 0.6068745


Test set: Avg. loss: 0.017526026407877603, AUC: 0.570713


Test set: Avg. loss: 0.05553884633382161, AUC: 0.42827850000000006


Test set: Avg. loss: 0.04969458897908529, AUC: 0.5678025000000001


Test set: Avg. loss: 0.01274657948811849, AUC: 0.5931595


Test set: Avg. loss: 0.10591863505045573, AUC: 0.5035


Test set: Avg


Test set: Avg. loss: 0.015886303265889485, AUC: 0.6142409999999999


Test set: Avg. loss: 0.01791955884297689, AUC: 0.591755


Test set: Avg. loss: 0.014516658147176106, AUC: 0.6145122499999999


Test set: Avg. loss: 0.014342881838480632, AUC: 0.6088125


Test set: Avg. loss: 0.013331167856852214, AUC: 0.61232225


Test set: Avg. loss: 0.09835511271158855, AUC: 0.5005


Test set: Avg. loss: 0.02358063824971517, AUC: 0.5811657500000001


Test set: Avg. loss: 0.019568593343098958, AUC: 0.5879745000000001


Test set: Avg. loss: 0.021043647130330405, AUC: 0.58303825


Test set: Avg. loss: 0.01741849199930827, AUC: 0.5906365


Test set: Avg. loss: 0.015072937647501627, AUC: 0.5977355


Test set: Avg. loss: 0.0319543399810791, AUC: 0.540517


Test set: Avg. loss: 0.01684540049235026, AUC: 0.5846134999999999


Test set: Avg. loss: 0.01591299883524577, AUC: 0.5831157499999999


Test set: Avg. loss: 0.014751671473185221, AUC: 0.587102


Test set: Avg. loss: 0.012039198557535807, AUC: 0.6045290


Test set: Avg. loss: 0.07394101206461588, AUC: 0.5244315


Test set: Avg. loss: 0.05383163833618164, AUC: 0.5218607500000001


Test set: Avg. loss: 0.047948720296223955, AUC: 0.5197680000000001


Test set: Avg. loss: 0.04567041397094727, AUC: 0.5183035


Test set: Avg. loss: 0.04384569549560547, AUC: 0.5168662500000001


Test set: Avg. loss: 0.052223325093587236, AUC: 0.41791325


Test set: Avg. loss: 0.0407930908203125, AUC: 0.5485535


Test set: Avg. loss: 0.03957231648763021, AUC: 0.5456650000000001


Test set: Avg. loss: 0.03849019495646159, AUC: 0.5431782500000001


Test set: Avg. loss: 0.03780831019083659, AUC: 0.5425665000000001


Test set: Avg. loss: 0.03719708506266276, AUC: 0.5456887500000001


Test set: Avg. loss: 0.1430635019938151, AUC: 0.4927605


Test set: Avg. loss: 0.0649750493367513, AUC: 0.5579395


Test set: Avg. loss: 0.05228489176432292, AUC: 0.54781225


Test set: Avg. loss: 0.04701221974690755, AUC: 0.54800525


Test set: Avg. loss: 0.04414041010538737, AUC: 0.

In [31]:
# Softmax 3 class oversampled

learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SoftmaxLogisticRegression(NUM_CLASSES_REDUCED, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.07714171091715495, AUC: 0.5059459999999999


Test set: Avg. loss: 0.026665886561075845, AUC: 0.73445625


Test set: Avg. loss: 0.0305110772450765, AUC: 0.72315275


Test set: Avg. loss: 0.13934913635253907, AUC: 0.5403140000000001


Test set: Avg. loss: 0.06933519744873047, AUC: 0.6295314999999999


Test set: Avg. loss: 0.027758974075317383, AUC: 0.7582260000000001


Test set: Avg. loss: 0.016065981864929198, AUC: 0.5326789999999999


Test set: Avg. loss: 0.05959995778401693, AUC: 0.5391515


Test set: Avg. loss: 0.06766993967692057, AUC: 0.6078125


Test set: Avg. loss: 0.10178028869628906, AUC: 0.5682477499999999


Test set: Avg. loss: 0.04995796330769857, AUC: 0.678669


Test set: Avg. loss: 0.039901153564453125, AUC: 0.67985275


Test set: Avg. loss: 0.03496906407674154, AUC: 0.5000150000000001


Test set: Avg. loss: 0.12870389048258463, AUC: 0.5654655


Test set: Avg. loss: 0.026147863388061524, AUC: 0.749375


Test set: Avg. loss: 0.02779640515645345, AUC:


Test set: Avg. loss: 0.009179949442545573, AUC: 0.70042775


Test set: Avg. loss: 0.009553075790405274, AUC: 0.6861422500000001


Test set: Avg. loss: 0.037033695220947266, AUC: 0.47265199999999996


Test set: Avg. loss: 0.014327217737833658, AUC: 0.6441334999999999


Test set: Avg. loss: 0.013015405972798665, AUC: 0.66257825


Test set: Avg. loss: 0.008705786069234212, AUC: 0.6634399999999999


Test set: Avg. loss: 0.008178181648254394, AUC: 0.6651017499999999


Test set: Avg. loss: 0.008094624360402425, AUC: 0.674504


Test set: Avg. loss: 0.07260543568929037, AUC: 0.5097507499999999


Test set: Avg. loss: 0.012593708992004394, AUC: 0.6775025


Test set: Avg. loss: 0.01021955172220866, AUC: 0.688895


Test set: Avg. loss: 0.008109107335408528, AUC: 0.66582575


Test set: Avg. loss: 0.009494519233703614, AUC: 0.6945859999999999


Test set: Avg. loss: 0.008523593266805013, AUC: 0.700994


Test set: Avg. loss: 0.04323506546020508, AUC: 0.5632995


Test set: Avg. loss: 0.010782269795735


Test set: Avg. loss: 0.025354435602823893, AUC: 0.43794900000000003


Test set: Avg. loss: 0.023398722330729167, AUC: 0.452291


Test set: Avg. loss: 0.021645716985066732, AUC: 0.46330899999999997


Test set: Avg. loss: 0.06269415283203125, AUC: 0.5039999999999999


Test set: Avg. loss: 0.01849854024251302, AUC: 0.628436


Test set: Avg. loss: 0.018249528249104818, AUC: 0.63708825


Test set: Avg. loss: 0.01789515813191732, AUC: 0.6388944999999999


Test set: Avg. loss: 0.017858828862508138, AUC: 0.644384


Test set: Avg. loss: 0.01768730672200521, AUC: 0.6506105


Test set: Avg. loss: 0.04042553456624349, AUC: 0.523719


Test set: Avg. loss: 0.026237008412679036, AUC: 0.58470725


Test set: Avg. loss: 0.02381073252360026, AUC: 0.586098


Test set: Avg. loss: 0.02227811559041341, AUC: 0.5949765


Test set: Avg. loss: 0.020931973139444986, AUC: 0.600526


Test set: Avg. loss: 0.020132923126220705, AUC: 0.6147785


Test set: Avg. loss: 0.021049159367879232, AUC: 0.5585137500000001


Tes

In [32]:
# Softmax 3 class weighted

learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SoftmaxLogisticRegression(NUM_CLASSES_REDUCED, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False, class_weights=weight)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["weighted", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.054066850026448565, AUC: 0.40514425


Test set: Avg. loss: 0.07813821919759115, AUC: 0.64722075


Test set: Avg. loss: 0.14005628458658853, AUC: 0.49749699999999997


Test set: Avg. loss: 0.051007169087727865, AUC: 0.7241285


Test set: Avg. loss: 0.030106971740722657, AUC: 0.7319955


Test set: Avg. loss: 0.12713991038004557, AUC: 0.59078


Test set: Avg. loss: 0.025873377482096355, AUC: 0.42918049999999996


Test set: Avg. loss: 0.07470969390869141, AUC: 0.674981


Test set: Avg. loss: 0.05992522811889649, AUC: 0.685105


Test set: Avg. loss: 0.05888475545247396, AUC: 0.59787225


Test set: Avg. loss: 0.08966105397542318, AUC: 0.652291


Test set: Avg. loss: 0.03542663828531901, AUC: 0.6904477499999999


Test set: Avg. loss: 0.14276482137044272, AUC: 0.49174450000000003


Test set: Avg. loss: 0.04319294357299805, AUC: 0.7434959999999999


Test set: Avg. loss: 0.0676743024190267, AUC: 0.6657135


Test set: Avg. loss: 0.03663051732381185, AUC: 0.7372555


Test s


Test set: Avg. loss: 0.008858798662821452, AUC: 0.66401


Test set: Avg. loss: 0.010971524556477865, AUC: 0.641126


Test set: Avg. loss: 0.01109793472290039, AUC: 0.70211875


Test set: Avg. loss: 0.00808847697575887, AUC: 0.68543675


Test set: Avg. loss: 0.008755759874979655, AUC: 0.68771125


Test set: Avg. loss: 0.11240451049804688, AUC: 0.4788595


Test set: Avg. loss: 0.009048368136088053, AUC: 0.6892889999999998


Test set: Avg. loss: 0.009703492482503255, AUC: 0.6993165


Test set: Avg. loss: 0.008245553016662598, AUC: 0.6788875000000001


Test set: Avg. loss: 0.010500175793965658, AUC: 0.6828694999999999


Test set: Avg. loss: 0.006432095686594645, AUC: 0.65953


Test set: Avg. loss: 0.17676974995930989, AUC: 0.49748875


Test set: Avg. loss: 0.01366906770070394, AUC: 0.6641900000000001


Test set: Avg. loss: 0.01599214744567871, AUC: 0.6715880000000001


Test set: Avg. loss: 0.007701399167378743, AUC: 0.6670919999999999


Test set: Avg. loss: 0.011733227094014486, AUC: 0.67


Test set: Avg. loss: 0.01949064572652181, AUC: 0.5046857499999999


Test set: Avg. loss: 0.01861827278137207, AUC: 0.524332


Test set: Avg. loss: 0.08791636912027995, AUC: 0.482536


Test set: Avg. loss: 0.018633498509724934, AUC: 0.490958


Test set: Avg. loss: 0.017163516998291016, AUC: 0.50155575


Test set: Avg. loss: 0.01667580477396647, AUC: 0.52344475


Test set: Avg. loss: 0.016185743967692057, AUC: 0.54220375


Test set: Avg. loss: 0.01564615281422933, AUC: 0.54919225


Test set: Avg. loss: 0.025237705866495767, AUC: 0.43169199999999996


Test set: Avg. loss: 0.02397275733947754, AUC: 0.49818375


Test set: Avg. loss: 0.02219730313618978, AUC: 0.527635


Test set: Avg. loss: 0.020898494720458984, AUC: 0.55243875


Test set: Avg. loss: 0.01914929262797038, AUC: 0.5668655


Test set: Avg. loss: 0.018058064778645834, AUC: 0.5826185


Test set: Avg. loss: 0.07199223581949869, AUC: 0.4509545


Test set: Avg. loss: 0.013369771321614583, AUC: 0.65537575


Test set: Avg. loss: 0.012

In [33]:
df = pd.DataFrame(rows, columns = col_names) 

In [34]:
print(df)

           name  num_classes classes_used         ratio  learning_rate  \
0        normal            2       (0, 1)        (1, 1)   1.000000e-05   
1        normal            2       (0, 1)        (1, 1)   1.000000e-06   
2        normal            2       (0, 1)        (1, 1)   5.000000e-07   
3        normal            2       (0, 1)        (1, 1)   1.000000e-07   
4        normal            2       (0, 1)        (1, 1)   1.000000e-08   
5         ratio            2       (0, 1)      (100, 1)   1.000000e-05   
6         ratio            2       (0, 1)      (100, 1)   1.000000e-06   
7         ratio            2       (0, 1)      (100, 1)   5.000000e-07   
8         ratio            2       (0, 1)      (100, 1)   1.000000e-07   
9         ratio            2       (0, 1)      (100, 1)   1.000000e-08   
10  oversampled            2       (0, 1)      (100, 1)   1.000000e-05   
11  oversampled            2       (0, 1)      (100, 1)   1.000000e-06   
12  oversampled            2       (0,

In [35]:
df.to_csv('results/auc_analysis')